In [20]:
from PIL import Image
import glob
import os
from pathlib import Path
#from tflite_model_maker import image_classifier, config
from matplotlib import pyplot as plt
import tensorflow as tf

In [21]:
tflite_interpreter = tf.lite.Interpreter(model_path='content/Models/model_fp16.tflite')
class_names = ["Cat", "Dog"]
tflite_interpreter.allocate_tensors()

In [22]:
def set_input_tensor(interpreter, image):
  """Sets the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image

In [23]:
def get_predictions(input_image):
    output_details = tflite_interpreter.get_output_details()
    set_input_tensor(tflite_interpreter, input_image)
    tflite_interpreter.invoke()
    tflite_model_prediction = tflite_interpreter.get_tensor(output_details[0]["index"])
    print(tflite_model_prediction)
    return tflite_model_prediction
    tflite_model_prediction = tflite_model_prediction.squeeze().argmax(axis = 0)
    pred_class = class_names[tflite_model_prediction]
    print(tflite_model_prediction)
    print(pred_class)
    #return pred_class 

In [24]:
samplePath="content/samples"
for f in os.listdir(samplePath):
    filename = os.path.join(samplePath,f)
    if os.path.isfile(filename):
        print(filename)
        img = tf.keras.preprocessing.image.load_img(filename , grayscale=False, color_mode='rgb', target_size=(224,224,3), interpolation='nearest')
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        result = get_predictions(input_image=img_array)
        if len(result) < 1 or len(result[0]) < 2:
            print("invalid result")
            continue
        if result[0][0] > 0.6 or result[0][1] > 0.6:
            print("its a cat/dog")
        else:
            print("nope")

content/samples/IMG_0085.JPG
[[0.8950261  0.10497386]]
its a cat/dog
content/samples/cat.JPG
[[0.94732946 0.05267052]]
its a cat/dog
content/samples/IMG_1766.JPG
[[0.90273315 0.09726681]]
its a cat/dog
content/samples/IMG_4274.JPG
[[0.570178 0.429822]]
nope
content/samples/IMG_4825.JPG
[[0.41351184 0.5864881 ]]
nope
content/samples/IMG_3379.JPG
[[0.76960105 0.23039901]]
its a cat/dog
content/samples/buns.jpg
[[0.5000508  0.49994922]]
nope
